In [13]:
#include <iostream>
#include <xtensor/xarray.hpp>
#include <xtensor/xio.hpp>
#include <xtensor/xview.hpp>
#include <xtensor/xarray.hpp>
#include <xtensor/xfixed.hpp>
#include <xtensor/xio.hpp>
#include <xtensor/xtensor.hpp>
#include <xtensor/xadapt.hpp>
#include <xtensor/xarray.hpp>
#include <xtensor/xaxis_slice_iterator.hpp>
#include <xtensor/xaxis_iterator.hpp>
#include <xtensor/xio.hpp>
#include <xtensor/xarray.hpp>
#include <xtensor/xindex_view.hpp>
#include <xtensor/xchunked_array.hpp>
#include <xtensor/xfixed.hpp>
#include <cstddef>
#include <vector>
#include <xtensor/xarray.hpp>
#include <xtensor/xadapt.hpp>
#include <istream>
#include <fstream>
#include <iostream>
#include <xtensor/xarray.hpp>
#include <xtensor/xcsv.hpp>
#include <iostream>
#include <fstream>
#include <functional>
#include <cstdio>
#include <xtensor/xcsv.hpp>
#include <xcpp/xdisplay.hpp>
#include <xtensor/xsort.hpp>

using namespace xt;
using namespace xt::placeholders;  // to enable _ syntax

#include <iostream>
#include <fstream>
using namespace std;

#include <chrono>
using namespace std::chrono;

In [14]:
double current_time(){
    auto current_time = std::chrono::system_clock::now();
    auto duration_in_seconds = std::chrono::duration<double>(current_time.time_since_epoch());
    double num_seconds = duration_in_seconds.count();
    return num_seconds;
}

## Similarity



In [15]:
float cosine_dissimilarity(const xarray<double>& a, const xarray<double>& b){ 
    auto numerator = sum(a * b);
    auto denominator = sqrt(sum(pow(a, 2))) * sqrt(sum(pow(b, 2)));
    auto result = numerator / denominator;
    return 1 - result(0);
}

## Normalization

In [16]:
auto normalize(const xarray<double>& a){ 
    xarray<double> result = a / sqrt(sum(pow(a, 2)));
    return result;
}

# Lecture dataset

In [29]:
ofstream outfile;
outfile.open("../out/LOG_dbscanrn_cpp_opt_lecture_dataset_D2_R12_k4_cosine_dissimilarity.log");

auto start =  chrono::high_resolution_clock::now();

ifstream input_file;
input_file.open ("../data/lecture_dataset.txt");
auto data = load_csv<double>(input_file);
input_file.close();

auto end =  chrono::high_resolution_clock::now();
duration<double> elapsed = (end - start);
outfile << std::setprecision (16) << current_time() << ",reading_data," <<
    "," << elapsed.count() << ",," << endl;

data.reshape(data.shape());
xarray<double> X = view(data, all(), range(0, 2));

In [30]:
xarray<double> CLUSTER; 
xarray<double> STATE;

In [31]:
auto clustering_algorithm(const xarray<float>& X_original, int k)
{
    outfile << std::setprecision (16) << current_time() << ",start log,,," << endl;
    
    // normalization
    auto start =  chrono::high_resolution_clock::now();
    xarray<float> X = normalize(X_original);
    auto end =  chrono::high_resolution_clock::now();
    duration<double> elapsed = (end - start);

    outfile << std::setprecision (16) << current_time() << ",normalization_time," <<
        "," << elapsed.count() << "," << endl;
    
    // each data point can be in one of 3 stages
    int NOT_VISITED = -1; // not visited point
    int VISITED = 0; // non-core point
    int CLUSTERED = 1; // core point

    // initial setup
    int n = X.shape()[0];
    xarray<double> cluster = ones<double>({n}) * -1; // cluster register
    xarray<double> state = ones<double>({n}) * NOT_VISITED; // state register
    int cluster_id = 1;
    xarray<int> all_point_indices = arange(n);
    map<int, vector<int>> point_rnn;
    map<int, xarray<int>> point_knn;
    
    xarray<int> all_point_sorted; 
    xarray<int> candidates;
    float real_max; 
    xarray<double> pessimistic_estimation;
    xarray<double> candidate_distance;
    int down_row;

    
    function < void( int ) > calc_pessimistic_estimation = [&] (
        int current_index 
    ) -> void {
        
        // choosing the next point to check if he is a better neighbor
        int new_candidate = all_point_sorted(down_row);
        bool previous_check = pessimistic_estimation(down_row) < real_max;
        down_row++;
        if (!previous_check) {
            point_knn[(int) current_index] = candidates;
            return;
        };
        
        if (previous_check) {
            float similarity = cosine_dissimilarity(
                new_candidate, all_point_sorted[current_index]
            );
            outfile << std::setprecision (16) << current_time() << ",similarity_calculation," << (int) current_index << ",1," << endl;
            if (similarity < real_max) {
                candidates = filter(candidates, not_equal(candidate_distance, real_max));
                candidates = concatenate(xtuple(candidates, xarray<int>{new_candidate}));
                candidate_distance = concatenate(xtuple(candidate_distance, xarray<double>{similarity}));
                real_max = amax(candidate_distance)(0);
                calc_pessimistic_estimation((int) current_index);
            };
        };
        calc_pessimistic_estimation((int) current_index);   
    };

    auto get_tiknn = [&] (int current_index, xarray<int> neighbor_indices, int k){
        
        auto start =  chrono::high_resolution_clock::now();
        vector<double> r_dist;
        for (auto& idx : all_point_indices) {
            r_dist.push_back(cosine_dissimilarity(row(X, idx), {0, 1}));
        };
        auto end = chrono::high_resolution_clock::now();
        duration<double> elapsed = end - start;
        
        // printf("%f,pessimistic_estimation_time,%d,%f,\n",current_time(),current_index ,elapsed.count());
        outfile << std::setprecision (16) << current_time() << ",pessimistic_estimation_time," <<
                current_index << "," << elapsed.count() << "," << endl;
     
        xarray<double> r_distance = adapt(r_dist, {r_dist.size()});
        auto current_index_r_distance = r_distance[current_index];
        
        start =  chrono::high_resolution_clock::now();
        pessimistic_estimation = abs(current_index_r_distance - r_distance);
        end = chrono::high_resolution_clock::now();
        elapsed = end - start;
        
        outfile << std::setprecision (16) << current_time() << ",dist_to_ref_point_time," <<
                current_index << "," << elapsed.count() << "," << endl;
        // printf("%f,dist_to_ref_point_time,%d,%f,\n",current_time(),current_index ,elapsed.count());
        
        start =  chrono::high_resolution_clock::now();
        xarray<int> sort_indices = argsort(pessimistic_estimation);
        all_point_sorted = index_view(all_point_indices, sort_indices); // df 
        end = chrono::high_resolution_clock::now();
        elapsed = end - start;
        
        outfile << std::setprecision (16) << current_time() << ",sorting_pessimistic_est_time," <<
                current_index << "," << elapsed.count() << "," << endl;
        // printf("%f,sorting_pessimistic_est_time,%d,%f,\n",current_time(),current_index ,elapsed.count());
        
        // selecting candidates for k - nearest neighbors
        candidates = view(all_point_sorted, range(1, k+1)); //dfx
        down_row = k + 2;
                
        // calculation of similarity for candidates
        vector<double> dist;
        for (auto& candidate_idx : candidates) {
            outfile << std::setprecision (16) << current_time() << ",similarity_calculation," << (int) current_index << ",1," << endl;
            dist.push_back(cosine_dissimilarity(row(X, current_index), row(X, candidate_idx)));
        }
        candidate_distance = adapt(dist, {dist.size()});
        real_max = amax(candidate_distance)(0);
        
        calc_pessimistic_estimation((int) current_index);
    };  
        
    auto get_knn = [&] (int current_index, xarray<int> neighbor_indices, int k){
        vector<float> neighbor_sim;
        for (auto& neighbor_index : neighbor_indices) {
            
            // outfile << std::setprecision (16) << current_time() << ",similarity_calculation," << (int) current_index << ",1," << endl;
            
            neighbor_sim.push_back(cosine_dissimilarity(row(X, neighbor_index), row(X, current_index)));
        };
        xarray<float> neighbor_similaritys = adapt(neighbor_sim, {neighbor_sim.size()});
        xarray<int> sort_indices = argsort(neighbor_similaritys);
        xarray<int> neighbor_indices_sorted = index_view(neighbor_indices, sort_indices); 
        point_knn[(int) current_index] = view(neighbor_indices_sorted, range(0, k));  
    };    
    
    auto get_pointwise_rnn = [&] (int current_index){
        vector<int> rnn;
        for (auto& neighbor_index : point_knn[(int) current_index]) {
            if ((point_knn[(int) neighbor_index], (int) current_index)) {
                rnn.push_back((int) neighbor_index);
            };
        };
        point_rnn[(int) current_index] = rnn;
    };
    
    auto get_rnn = [&] (){
        auto start =  chrono::high_resolution_clock::now();
        for (int current_index = 0; current_index < n; current_index++) {
            // remove current index from neighbors
            xarray<int> neighbor_indices = col(
                from_indices(argwhere(not_equal(all_point_indices, current_index))), 0
            );
            get_tiknn((int) current_index, neighbor_indices, k);
            
            // printf("%f,knn_neighbors_id,%d,,",current_time(), current_index);
            outfile << std::setprecision (16) << current_time() << ",knn_neighbors_id," <<
                    current_index << ",,";
            for(auto& i : point_knn[(int) current_index]) {outfile << i << ";"; }
            outfile << endl;
            
            // for(auto& i : point_knn[(int) current_index]) {std::cout << i << ";"; }
            // printf("\n");
            // printf("%f,|knn_neighbors|,%d,%d,\n",current_time(), current_index, (int) point_knn[(int) current_index].shape(0));
            outfile << std::setprecision (16) << current_time() << ",|knn_neighbors|," <<
                    current_index << "," << (int) point_knn[(int) current_index].shape(0) << "," << endl;
        };
        auto end = chrono::high_resolution_clock::now();
        duration<double> elapsed = end - start;
        
        outfile << std::setprecision (16) << current_time() << ",knn_time," <<
            ",," << elapsed.count() << ',' << endl;
        // printf("%f,knn_time,,%f,\n",current_time(), elapsed.count());
        
        start =  chrono::high_resolution_clock::now();
        for (int current_index =  0; current_index < n; current_index++) {
            get_pointwise_rnn((int) current_index);
            
            // printf("%f,rnn_neighbors_id,%d,,",current_time(), current_index);
            outfile << std::setprecision (16) << current_time() << ",rnn_neighbors_id," <<
                    current_index << ",,";
            for(auto& i : point_knn[(int) current_index]) {outfile << i << ";"; }
            outfile << endl;
            
            // for(auto& i : point_knn[(int) current_index]) {std::cout << i << ";"; }
            // printf("\n");
            // printf("%f,|rnn_neighbors|,%d,%d,\n",current_time(), current_index, (int) point_knn[(int) current_index].shape(0));
            outfile << std::setprecision (16) << current_time() << ",|rnn_neighbors|," <<
                    current_index << "," << (int) point_knn[(int) current_index].shape(0) << "," << endl;
        };
        end = chrono::high_resolution_clock::now();
        elapsed = end - start;
        outfile << std::setprecision (16) << current_time() << ",rnn_time," <<
            "," << elapsed.count() << ',' << endl;
    };

    // extend cluster
    function < void( int ) > search = [&] (int current_index) -> void {
        if (point_rnn[(int) current_index].size() <  k) {
            state[(int) current_index] = VISITED;
        } else {
            state[(int) current_index] = CLUSTERED;
            cluster[(int) current_index] = cluster_id;
            for (auto& neighbor_index : point_rnn[(int) current_index]) {
                if (state[(int) neighbor_index] == NOT_VISITED) {
                    search(neighbor_index);
                };
                state[(int) neighbor_index] = CLUSTERED;
                cluster[(int) neighbor_index] = cluster_id;
            };
        };
    };
    
    get_rnn();
    
    // visit all points
    while (any(equal(state, NOT_VISITED))){
        xarray<int> idx = from_indices(argwhere(equal(state, NOT_VISITED)));
        search(idx(0,0));
        cluster_id++;
    }

    // clusterize all outlier points to nearest cluster 
    while (any(equal(state, VISITED))){
        xarray<int> idx_not_clustered = from_indices(argwhere(equal(state, VISITED)));
        xarray<int> idx_clustered = from_indices(argwhere(equal(state, CLUSTERED)));
        get_knn((int) idx_not_clustered(0,0), idx_clustered, 1);
        auto closest_clustered_idx = point_knn[(int) idx_not_clustered(0,0)](0);
        cluster[(int) idx_not_clustered(0,0)] = cluster[(int) closest_clustered_idx];
        state[(int) idx_not_clustered(0,0)] = CLUSTERED;
    }
    outfile << std::setprecision (16) << current_time() << ",stop log,,," << endl;
    CLUSTER = cluster;
    STATE = state;
    // return xtuple(cluster, );
}

In [32]:
clustering_algorithm(X, 4);

## Save output to csv

In [33]:
start = chrono::high_resolution_clock::now();

ofstream out_file;
out_file.open("../out/dbscanrn_opt_cpp_lecture_dataset.csv");
dump_csv(out_file, stack(xtuple(CLUSTER, STATE), 1));
out_file.close();

duration<double> elapsed = (chrono::high_resolution_clock::now() - start);
outfile << std::setprecision (16) << current_time() << ",writing_data," <<
    "," << elapsed.count() << "," << endl;
outfile.close();

# Compound dataset

In [48]:
ofstream outfile;
outfile.open("../out/LOG_dbscanrn_opt_cpp_compound_dataset_D2_R399_k6_cosine_dissimilarity_r01.log");

auto start =  chrono::high_resolution_clock::now();

ifstream input_file;
// input_file.open ("../data/lecture_dataset.txt");
input_file.open ("../data/compound_dataset.txt");
// input_file.open ("../data/flame_dataset.txt");
auto data = load_csv<double>(input_file);
input_file.close();

auto end =  chrono::high_resolution_clock::now();
duration<double> elapsed = (end - start);
outfile << std::setprecision (16) << current_time() << ",reading_data," <<
    "," << elapsed.count() << ",," << endl;

data.reshape(data.shape());
xarray<double> X = view(data, all(), range(0, 2));

In [49]:
xarray<double> CLUSTER; 
xarray<double> STATE;

In [50]:
auto clustering_algorithm(const xarray<float>& X_original, int k)
{
    outfile << std::setprecision (16) << current_time() << ",start log,,," << endl;
    
    // normalization
    auto start =  chrono::high_resolution_clock::now();
    xarray<float> X = normalize(X_original);
    auto end =  chrono::high_resolution_clock::now();
    duration<double> elapsed = (end - start);

    outfile << std::setprecision (16) << current_time() << ",normalization_time," <<
        "," << elapsed.count() << "," << endl;
    
    // each data point can be in one of 3 stages
    int NOT_VISITED = -1; // not visited point
    int VISITED = 0; // non-core point
    int CLUSTERED = 1; // core point

    // initial setup
    int n = X.shape()[0];
    xarray<double> cluster = ones<double>({n}) * -1; // cluster register
    xarray<double> state = ones<double>({n}) * NOT_VISITED; // state register
    int cluster_id = 1;
    xarray<int> all_point_indices = arange(n);
    map<int, vector<int>> point_rnn;
    map<int, xarray<int>> point_knn;
    
    xarray<int> all_point_sorted; 
    xarray<int> candidates;
    float real_max; 
    xarray<double> pessimistic_estimation;
    xarray<double> candidate_distance;
    int down_row;

    
    function < void( int ) > calc_pessimistic_estimation = [&] (
        int current_index 
    ) -> void {
        
        // choosing the next point to check if he is a better neighbor
        int new_candidate = all_point_sorted(down_row);
        bool previous_check = pessimistic_estimation(down_row) < real_max;
        down_row++;
        if (!previous_check) {
            point_knn[(int) current_index] = candidates;
            return;
        };
        
        if (previous_check) {
            float similarity = cosine_dissimilarity(
                new_candidate, all_point_sorted[current_index]
            );
            outfile << std::setprecision (16) << current_time() << ",similarity_calculation," << (int) current_index << ",1," << endl;
            if (similarity < real_max) {
                candidates = filter(candidates, not_equal(candidate_distance, real_max));
                candidates = concatenate(xtuple(candidates, xarray<int>{new_candidate}));
                candidate_distance = concatenate(xtuple(candidate_distance, xarray<double>{similarity}));
                real_max = amax(candidate_distance)(0);
                calc_pessimistic_estimation((int) current_index);
            };
        };
        calc_pessimistic_estimation((int) current_index);   
    };

    auto get_tiknn = [&] (int current_index, xarray<int> neighbor_indices, int k){
        
        auto start =  chrono::high_resolution_clock::now();
        vector<double> r_dist;
        for (auto& idx : all_point_indices) {
            r_dist.push_back(cosine_dissimilarity(row(X, idx), {0, 1}));
        };
        auto end = chrono::high_resolution_clock::now();
        duration<double> elapsed = end - start;
        
        // printf("%f,pessimistic_estimation_time,%d,%f,\n",current_time(),current_index ,elapsed.count());
        outfile << std::setprecision (16) << current_time() << ",pessimistic_estimation_time," <<
                current_index << "," << elapsed.count() << "," << endl;
     
        xarray<double> r_distance = adapt(r_dist, {r_dist.size()});
        auto current_index_r_distance = r_distance[current_index];
        
        start =  chrono::high_resolution_clock::now();
        pessimistic_estimation = abs(current_index_r_distance - r_distance);
        end = chrono::high_resolution_clock::now();
        elapsed = end - start;
        
        outfile << std::setprecision (16) << current_time() << ",dist_to_ref_point_time," <<
                current_index << "," << elapsed.count() << "," << endl;
        // printf("%f,dist_to_ref_point_time,%d,%f,\n",current_time(),current_index ,elapsed.count());
        
        start =  chrono::high_resolution_clock::now();
        xarray<int> sort_indices = argsort(pessimistic_estimation);
        all_point_sorted = index_view(all_point_indices, sort_indices); // df 
        end = chrono::high_resolution_clock::now();
        elapsed = end - start;
        
        outfile << std::setprecision (16) << current_time() << ",sorting_pessimistic_est_time," <<
                current_index << "," << elapsed.count() << "," << endl;
        // printf("%f,sorting_pessimistic_est_time,%d,%f,\n",current_time(),current_index ,elapsed.count());
        
        // selecting candidates for k - nearest neighbors
        candidates = view(all_point_sorted, range(1, k+1)); //dfx
        down_row = k + 2;
                
        // calculation of similarity for candidates
        vector<double> dist;
        for (auto& candidate_idx : candidates) {
            outfile << std::setprecision (16) << current_time() << ",similarity_calculation," << (int) current_index << ",1," << endl;
            dist.push_back(cosine_dissimilarity(row(X, current_index), row(X, candidate_idx)));
        }
        candidate_distance = adapt(dist, {dist.size()});
        real_max = amax(candidate_distance)(0);
        
        calc_pessimistic_estimation((int) current_index);
    };  
        
    auto get_knn = [&] (int current_index, xarray<int> neighbor_indices, int k){
        vector<float> neighbor_sim;
        for (auto& neighbor_index : neighbor_indices) {
            
            // outfile << std::setprecision (16) << current_time() << ",similarity_calculation," << (int) current_index << ",1," << endl;
            
            neighbor_sim.push_back(cosine_dissimilarity(row(X, neighbor_index), row(X, current_index)));
        };
        xarray<float> neighbor_similaritys = adapt(neighbor_sim, {neighbor_sim.size()});
        xarray<int> sort_indices = argsort(neighbor_similaritys);
        xarray<int> neighbor_indices_sorted = index_view(neighbor_indices, sort_indices); 
        point_knn[(int) current_index] = view(neighbor_indices_sorted, range(0, k));  
    };    
    
    auto get_pointwise_rnn = [&] (int current_index){
        vector<int> rnn;
        for (auto& neighbor_index : point_knn[(int) current_index]) {
            if ((point_knn[(int) neighbor_index], (int) current_index)) {
                rnn.push_back((int) neighbor_index);
            };
        };
        point_rnn[(int) current_index] = rnn;
    };
    
    auto get_rnn = [&] (){
        auto start =  chrono::high_resolution_clock::now();
        for (int current_index = 0; current_index < n; current_index++) {
            // remove current index from neighbors
            xarray<int> neighbor_indices = col(
                from_indices(argwhere(not_equal(all_point_indices, current_index))), 0
            );
            get_tiknn((int) current_index, neighbor_indices, k);
            
            // printf("%f,knn_neighbors_id,%d,,",current_time(), current_index);
            outfile << std::setprecision (16) << current_time() << ",knn_neighbors_id," <<
                    current_index << ",,";
            for(auto& i : point_knn[(int) current_index]) {outfile << i << ";"; }
            outfile << endl;
            
            // for(auto& i : point_knn[(int) current_index]) {std::cout << i << ";"; }
            // printf("\n");
            // printf("%f,|knn_neighbors|,%d,%d,\n",current_time(), current_index, (int) point_knn[(int) current_index].shape(0));
            outfile << std::setprecision (16) << current_time() << ",|knn_neighbors|," <<
                    current_index << "," << (int) point_knn[(int) current_index].shape(0) << "," << endl;
        };
        auto end = chrono::high_resolution_clock::now();
        duration<double> elapsed = end - start;
        
        outfile << std::setprecision (16) << current_time() << ",knn_time," <<
            ",," << elapsed.count() << ',' << endl;
        // printf("%f,knn_time,,%f,\n",current_time(), elapsed.count());
        
        start =  chrono::high_resolution_clock::now();
        for (int current_index =  0; current_index < n; current_index++) {
            get_pointwise_rnn((int) current_index);
            
            // printf("%f,rnn_neighbors_id,%d,,",current_time(), current_index);
            outfile << std::setprecision (16) << current_time() << ",rnn_neighbors_id," <<
                    current_index << ",,";
            for(auto& i : point_knn[(int) current_index]) {outfile << i << ";"; }
            outfile << endl;
            
            // for(auto& i : point_knn[(int) current_index]) {std::cout << i << ";"; }
            // printf("\n");
            // printf("%f,|rnn_neighbors|,%d,%d,\n",current_time(), current_index, (int) point_knn[(int) current_index].shape(0));
            outfile << std::setprecision (16) << current_time() << ",|rnn_neighbors|," <<
                    current_index << "," << (int) point_knn[(int) current_index].shape(0) << "," << endl;
        };
        end = chrono::high_resolution_clock::now();
        elapsed = end - start;
        outfile << std::setprecision (16) << current_time() << ",rnn_time," <<
            "," << elapsed.count() << ',' << endl;
    };

    // extend cluster
    function < void( int ) > search = [&] (int current_index) -> void {
        if (point_rnn[(int) current_index].size() <  k) {
            state[(int) current_index] = VISITED;
        } else {
            state[(int) current_index] = CLUSTERED;
            cluster[(int) current_index] = cluster_id;
            for (auto& neighbor_index : point_rnn[(int) current_index]) {
                if (state[(int) neighbor_index] == NOT_VISITED) {
                    search(neighbor_index);
                };
                state[(int) neighbor_index] = CLUSTERED;
                cluster[(int) neighbor_index] = cluster_id;
            };
        };
    };
    
    get_rnn();
    
    // visit all points
    while (any(equal(state, NOT_VISITED))){
        xarray<int> idx = from_indices(argwhere(equal(state, NOT_VISITED)));
        search(idx(0,0));
        cluster_id++;
    }

    // clusterize all outlier points to nearest cluster 
    while (any(equal(state, VISITED))){
        xarray<int> idx_not_clustered = from_indices(argwhere(equal(state, VISITED)));
        xarray<int> idx_clustered = from_indices(argwhere(equal(state, CLUSTERED)));
        get_knn((int) idx_not_clustered(0,0), idx_clustered, 1);
        auto closest_clustered_idx = point_knn[(int) idx_not_clustered(0,0)](0);
        cluster[(int) idx_not_clustered(0,0)] = cluster[(int) closest_clustered_idx];
        state[(int) idx_not_clustered(0,0)] = CLUSTERED;
    }
    outfile << std::setprecision (16) << current_time() << ",stop log,,," << endl;
    CLUSTER = cluster;
    STATE = state;
    // return xtuple(cluster, );
}

In [51]:
clustering_algorithm(X, 6);

## Save output to csv

In [52]:
start = chrono::high_resolution_clock::now();

ofstream out_file;
out_file.open("../out/dbscanrn_opt_cpp_compound_dataset.csv");
dump_csv(out_file, stack(xtuple(CLUSTER, STATE), 1));
out_file.close();

// end =  chrono::high_resolution_clock::now();
duration<double> elapsed = (chrono::high_resolution_clock::now() - start);
outfile << std::setprecision (16) << current_time() << ",writing_data," <<
    "," << elapsed.count() << "," << endl;
outfile.close();

# Jain dataset

In [67]:
ofstream outfile;
outfile.open("../out/LOG_dbscanrn_opt_cpp_jain_dataset_D2_R399_k7_cosine_dissimilarity_r01.log");

auto start =  chrono::high_resolution_clock::now();

ifstream input_file;
input_file.open ("../data/jain_dataset.txt");
auto data = load_csv<double>(input_file);
input_file.close();

auto end =  chrono::high_resolution_clock::now();
duration<double> elapsed = (end - start);
outfile << std::setprecision (16) << current_time() << ",reading_data," <<
    "," << elapsed.count() << ",," << endl;

data.reshape(data.shape());
xarray<double> X = view(data, all(), range(0, 2));

In [68]:
xarray<double> CLUSTER; 
xarray<double> STATE;

In [69]:
auto clustering_algorithm(const xarray<float>& X_original, int k)
{
    outfile << std::setprecision (16) << current_time() << ",start log,,," << endl;
    
    // normalization
    auto start =  chrono::high_resolution_clock::now();
    xarray<float> X = normalize(X_original);
    auto end =  chrono::high_resolution_clock::now();
    duration<double> elapsed = (end - start);

    outfile << std::setprecision (16) << current_time() << ",normalization_time," <<
        "," << elapsed.count() << "," << endl;
    
    // each data point can be in one of 3 stages
    int NOT_VISITED = -1; // not visited point
    int VISITED = 0; // non-core point
    int CLUSTERED = 1; // core point

    // initial setup
    int n = X.shape()[0];
    xarray<double> cluster = ones<double>({n}) * -1; // cluster register
    xarray<double> state = ones<double>({n}) * NOT_VISITED; // state register
    int cluster_id = 1;
    xarray<int> all_point_indices = arange(n);
    map<int, vector<int>> point_rnn;
    map<int, xarray<int>> point_knn;
    
    xarray<int> all_point_sorted; 
    xarray<int> candidates;
    float real_max; 
    xarray<double> pessimistic_estimation;
    xarray<double> candidate_distance;
    int down_row;

    
    function < void( int ) > calc_pessimistic_estimation = [&] (
        int current_index 
    ) -> void {
        
        // choosing the next point to check if he is a better neighbor
        int new_candidate = all_point_sorted(down_row);
        bool previous_check = pessimistic_estimation(down_row) < real_max;
        down_row++;
        if (!previous_check) {
            point_knn[(int) current_index] = candidates;
            return;
        };
        
        if (previous_check) {
            float similarity = cosine_dissimilarity(
                new_candidate, all_point_sorted[current_index]
            );
            outfile << std::setprecision (16) << current_time() << ",similarity_calculation," << (int) current_index << ",1," << endl;
            if (similarity < real_max) {
                candidates = filter(candidates, not_equal(candidate_distance, real_max));
                candidates = concatenate(xtuple(candidates, xarray<int>{new_candidate}));
                candidate_distance = concatenate(xtuple(candidate_distance, xarray<double>{similarity}));
                real_max = amax(candidate_distance)(0);
                calc_pessimistic_estimation((int) current_index);
            };
        };
        calc_pessimistic_estimation((int) current_index);   
    };

    auto get_tiknn = [&] (int current_index, xarray<int> neighbor_indices, int k){
        
        auto start =  chrono::high_resolution_clock::now();
        vector<double> r_dist;
        for (auto& idx : all_point_indices) {
            r_dist.push_back(cosine_dissimilarity(row(X, idx), {0, 1}));
        };
        auto end = chrono::high_resolution_clock::now();
        duration<double> elapsed = end - start;
        
        // printf("%f,pessimistic_estimation_time,%d,%f,\n",current_time(),current_index ,elapsed.count());
        outfile << std::setprecision (16) << current_time() << ",pessimistic_estimation_time," <<
                current_index << "," << elapsed.count() << "," << endl;
     
        xarray<double> r_distance = adapt(r_dist, {r_dist.size()});
        auto current_index_r_distance = r_distance[current_index];
        
        start =  chrono::high_resolution_clock::now();
        pessimistic_estimation = abs(current_index_r_distance - r_distance);
        end = chrono::high_resolution_clock::now();
        elapsed = end - start;
        
        outfile << std::setprecision (16) << current_time() << ",dist_to_ref_point_time," <<
                current_index << "," << elapsed.count() << "," << endl;
        // printf("%f,dist_to_ref_point_time,%d,%f,\n",current_time(),current_index ,elapsed.count());
        
        start =  chrono::high_resolution_clock::now();
        xarray<int> sort_indices = argsort(pessimistic_estimation);
        all_point_sorted = index_view(all_point_indices, sort_indices); // df 
        end = chrono::high_resolution_clock::now();
        elapsed = end - start;
        
        outfile << std::setprecision (16) << current_time() << ",sorting_pessimistic_est_time," <<
                current_index << "," << elapsed.count() << "," << endl;
        // printf("%f,sorting_pessimistic_est_time,%d,%f,\n",current_time(),current_index ,elapsed.count());
        
        // selecting candidates for k - nearest neighbors
        candidates = view(all_point_sorted, range(1, k+1)); //dfx
        down_row = k + 2;
                
        // calculation of similarity for candidates
        vector<double> dist;
        for (auto& candidate_idx : candidates) {
            outfile << std::setprecision (16) << current_time() << ",similarity_calculation," << (int) current_index << ",1," << endl;
            dist.push_back(cosine_dissimilarity(row(X, current_index), row(X, candidate_idx)));
        }
        candidate_distance = adapt(dist, {dist.size()});
        real_max = amax(candidate_distance)(0);
        
        calc_pessimistic_estimation((int) current_index);
    };  
        
    auto get_knn = [&] (int current_index, xarray<int> neighbor_indices, int k){
        vector<float> neighbor_sim;
        for (auto& neighbor_index : neighbor_indices) {
            
            // outfile << std::setprecision (16) << current_time() << ",similarity_calculation," << (int) current_index << ",1," << endl;
            
            neighbor_sim.push_back(cosine_dissimilarity(row(X, neighbor_index), row(X, current_index)));
        };
        xarray<float> neighbor_similaritys = adapt(neighbor_sim, {neighbor_sim.size()});
        xarray<int> sort_indices = argsort(neighbor_similaritys);
        xarray<int> neighbor_indices_sorted = index_view(neighbor_indices, sort_indices); 
        point_knn[(int) current_index] = view(neighbor_indices_sorted, range(0, k));  
    };    
    
    auto get_pointwise_rnn = [&] (int current_index){
        vector<int> rnn;
        for (auto& neighbor_index : point_knn[(int) current_index]) {
            if ((point_knn[(int) neighbor_index], (int) current_index)) {
                rnn.push_back((int) neighbor_index);
            };
        };
        point_rnn[(int) current_index] = rnn;
    };
    
    auto get_rnn = [&] (){
        auto start =  chrono::high_resolution_clock::now();
        for (int current_index = 0; current_index < n; current_index++) {
            // remove current index from neighbors
            xarray<int> neighbor_indices = col(
                from_indices(argwhere(not_equal(all_point_indices, current_index))), 0
            );
            get_tiknn((int) current_index, neighbor_indices, k);
            
            // printf("%f,knn_neighbors_id,%d,,",current_time(), current_index);
            outfile << std::setprecision (16) << current_time() << ",knn_neighbors_id," <<
                    current_index << ",,";
            for(auto& i : point_knn[(int) current_index]) {outfile << i << ";"; }
            outfile << endl;
            
            // for(auto& i : point_knn[(int) current_index]) {std::cout << i << ";"; }
            // printf("\n");
            // printf("%f,|knn_neighbors|,%d,%d,\n",current_time(), current_index, (int) point_knn[(int) current_index].shape(0));
            outfile << std::setprecision (16) << current_time() << ",|knn_neighbors|," <<
                    current_index << "," << (int) point_knn[(int) current_index].shape(0) << "," << endl;
        };
        auto end = chrono::high_resolution_clock::now();
        duration<double> elapsed = end - start;
        
        outfile << std::setprecision (16) << current_time() << ",knn_time," <<
            ",," << elapsed.count() << ',' << endl;
        // printf("%f,knn_time,,%f,\n",current_time(), elapsed.count());
        
        start =  chrono::high_resolution_clock::now();
        for (int current_index =  0; current_index < n; current_index++) {
            get_pointwise_rnn((int) current_index);
            
            // printf("%f,rnn_neighbors_id,%d,,",current_time(), current_index);
            outfile << std::setprecision (16) << current_time() << ",rnn_neighbors_id," <<
                    current_index << ",,";
            for(auto& i : point_knn[(int) current_index]) {outfile << i << ";"; }
            outfile << endl;
            
            // for(auto& i : point_knn[(int) current_index]) {std::cout << i << ";"; }
            // printf("\n");
            // printf("%f,|rnn_neighbors|,%d,%d,\n",current_time(), current_index, (int) point_knn[(int) current_index].shape(0));
            outfile << std::setprecision (16) << current_time() << ",|rnn_neighbors|," <<
                    current_index << "," << (int) point_knn[(int) current_index].shape(0) << "," << endl;
        };
        end = chrono::high_resolution_clock::now();
        elapsed = end - start;
        outfile << std::setprecision (16) << current_time() << ",rnn_time," <<
            "," << elapsed.count() << ',' << endl;
    };

    // extend cluster
    function < void( int ) > search = [&] (int current_index) -> void {
        if (point_rnn[(int) current_index].size() <  k) {
            state[(int) current_index] = VISITED;
        } else {
            state[(int) current_index] = CLUSTERED;
            cluster[(int) current_index] = cluster_id;
            for (auto& neighbor_index : point_rnn[(int) current_index]) {
                if (state[(int) neighbor_index] == NOT_VISITED) {
                    search(neighbor_index);
                };
                state[(int) neighbor_index] = CLUSTERED;
                cluster[(int) neighbor_index] = cluster_id;
            };
        };
    };
    
    get_rnn();
    
    // visit all points
    while (any(equal(state, NOT_VISITED))){
        xarray<int> idx = from_indices(argwhere(equal(state, NOT_VISITED)));
        search(idx(0,0));
        cluster_id++;
    }

    // clusterize all outlier points to nearest cluster 
    while (any(equal(state, VISITED))){
        xarray<int> idx_not_clustered = from_indices(argwhere(equal(state, VISITED)));
        xarray<int> idx_clustered = from_indices(argwhere(equal(state, CLUSTERED)));
        get_knn((int) idx_not_clustered(0,0), idx_clustered, 1);
        auto closest_clustered_idx = point_knn[(int) idx_not_clustered(0,0)](0);
        cluster[(int) idx_not_clustered(0,0)] = cluster[(int) closest_clustered_idx];
        state[(int) idx_not_clustered(0,0)] = CLUSTERED;
    }
    outfile << std::setprecision (16) << current_time() << ",stop log,,," << endl;
    CLUSTER = cluster;
    STATE = state;
    // return xtuple(cluster, );
}

In [70]:
clustering_algorithm(X, 7);

## Save output to csv

In [71]:
start = chrono::high_resolution_clock::now();

ofstream out_file;
out_file.open("../out/dbscanrn_opt_cpp_jain_dataset.csv");
dump_csv(out_file, stack(xtuple(CLUSTER, STATE), 1));
out_file.close();

duration<double> elapsed = (chrono::high_resolution_clock::now() - start);
outfile << std::setprecision (16) << current_time() << ",writing_data," <<
    "," << elapsed.count() << "," << endl;
outfile.close();

# Flame dataset

In [90]:
ofstream outfile;
outfile.open("../out/LOG_dbscanrn_opt_cpp_flame_dataset_D2_R240_k7_cosine_dissimilarity_r01.log");

auto start =  chrono::high_resolution_clock::now();

ifstream input_file;
input_file.open ("../data/flame_dataset.txt");
auto data = load_csv<double>(input_file);
input_file.close();

auto end =  chrono::high_resolution_clock::now();
duration<double> elapsed = (end - start);
outfile << std::setprecision (16) << current_time() << ",reading_data," <<
    "," << elapsed.count() << ",," << endl;

data.reshape(data.shape());
xarray<double> X = view(data, all(), range(0, 2));

In [91]:
xarray<double> CLUSTER; 
xarray<double> STATE;

In [92]:
auto clustering_algorithm(const xarray<float>& X_original, int k)
{
    outfile << std::setprecision (16) << current_time() << ",start log,,," << endl;
    
    // normalization
    auto start =  chrono::high_resolution_clock::now();
    xarray<float> X = normalize(X_original);
    auto end =  chrono::high_resolution_clock::now();
    duration<double> elapsed = (end - start);

    outfile << std::setprecision (16) << current_time() << ",normalization_time," <<
        "," << elapsed.count() << "," << endl;
    
    // each data point can be in one of 3 stages
    int NOT_VISITED = -1; // not visited point
    int VISITED = 0; // non-core point
    int CLUSTERED = 1; // core point

    // initial setup
    int n = X.shape()[0];
    xarray<double> cluster = ones<double>({n}) * -1; // cluster register
    xarray<double> state = ones<double>({n}) * NOT_VISITED; // state register
    int cluster_id = 1;
    xarray<int> all_point_indices = arange(n);
    map<int, vector<int>> point_rnn;
    map<int, xarray<int>> point_knn;
    
    xarray<int> all_point_sorted; 
    xarray<int> candidates;
    float real_max; 
    xarray<double> pessimistic_estimation;
    xarray<double> candidate_distance;
    int down_row;

    
    function < void( int ) > calc_pessimistic_estimation = [&] (
        int current_index 
    ) -> void {
        
        // choosing the next point to check if he is a better neighbor
        int new_candidate = all_point_sorted(down_row);
        bool previous_check = pessimistic_estimation(down_row) < real_max;
        down_row++;
        if (!previous_check) {
            point_knn[(int) current_index] = candidates;
            return;
        };
        
        if (previous_check) {
            float similarity = cosine_dissimilarity(
                new_candidate, all_point_sorted[current_index]
            );
            outfile << std::setprecision (16) << current_time() << ",similarity_calculation," << (int) current_index << ",1," << endl;
            if (similarity < real_max) {
                candidates = filter(candidates, not_equal(candidate_distance, real_max));
                candidates = concatenate(xtuple(candidates, xarray<int>{new_candidate}));
                candidate_distance = concatenate(xtuple(candidate_distance, xarray<double>{similarity}));
                real_max = amax(candidate_distance)(0);
                calc_pessimistic_estimation((int) current_index);
            };
        };
        calc_pessimistic_estimation((int) current_index);   
    };

    auto get_tiknn = [&] (int current_index, xarray<int> neighbor_indices, int k){
        
        auto start =  chrono::high_resolution_clock::now();
        vector<double> r_dist;
        for (auto& idx : all_point_indices) {
            r_dist.push_back(cosine_dissimilarity(row(X, idx), {0, 1}));
        };
        auto end = chrono::high_resolution_clock::now();
        duration<double> elapsed = end - start;
        
        // printf("%f,pessimistic_estimation_time,%d,%f,\n",current_time(),current_index ,elapsed.count());
        outfile << std::setprecision (16) << current_time() << ",pessimistic_estimation_time," <<
                current_index << "," << elapsed.count() << "," << endl;
     
        xarray<double> r_distance = adapt(r_dist, {r_dist.size()});
        auto current_index_r_distance = r_distance[current_index];
        
        start =  chrono::high_resolution_clock::now();
        pessimistic_estimation = abs(current_index_r_distance - r_distance);
        end = chrono::high_resolution_clock::now();
        elapsed = end - start;
        
        outfile << std::setprecision (16) << current_time() << ",dist_to_ref_point_time," <<
                current_index << "," << elapsed.count() << "," << endl;
        // printf("%f,dist_to_ref_point_time,%d,%f,\n",current_time(),current_index ,elapsed.count());
        
        start =  chrono::high_resolution_clock::now();
        xarray<int> sort_indices = argsort(pessimistic_estimation);
        all_point_sorted = index_view(all_point_indices, sort_indices); // df 
        end = chrono::high_resolution_clock::now();
        elapsed = end - start;
        
        outfile << std::setprecision (16) << current_time() << ",sorting_pessimistic_est_time," <<
                current_index << "," << elapsed.count() << "," << endl;
        // printf("%f,sorting_pessimistic_est_time,%d,%f,\n",current_time(),current_index ,elapsed.count());
        
        // selecting candidates for k - nearest neighbors
        candidates = view(all_point_sorted, range(1, k+1)); //dfx
        down_row = k + 2;
                
        // calculation of similarity for candidates
        vector<double> dist;
        for (auto& candidate_idx : candidates) {
            outfile << std::setprecision (16) << current_time() << ",similarity_calculation," << (int) current_index << ",1," << endl;
            dist.push_back(cosine_dissimilarity(row(X, current_index), row(X, candidate_idx)));
        }
        candidate_distance = adapt(dist, {dist.size()});
        real_max = amax(candidate_distance)(0);
        
        calc_pessimistic_estimation((int) current_index);
    };  
        
    auto get_knn = [&] (int current_index, xarray<int> neighbor_indices, int k){
        vector<float> neighbor_sim;
        for (auto& neighbor_index : neighbor_indices) {
            
            // outfile << std::setprecision (16) << current_time() << ",similarity_calculation," << (int) current_index << ",1," << endl;
            
            neighbor_sim.push_back(cosine_dissimilarity(row(X, neighbor_index), row(X, current_index)));
        };
        xarray<float> neighbor_similaritys = adapt(neighbor_sim, {neighbor_sim.size()});
        xarray<int> sort_indices = argsort(neighbor_similaritys);
        xarray<int> neighbor_indices_sorted = index_view(neighbor_indices, sort_indices); 
        point_knn[(int) current_index] = view(neighbor_indices_sorted, range(0, k));  
    };    
    
    auto get_pointwise_rnn = [&] (int current_index){
        vector<int> rnn;
        for (auto& neighbor_index : point_knn[(int) current_index]) {
            if ((point_knn[(int) neighbor_index], (int) current_index)) {
                rnn.push_back((int) neighbor_index);
            };
        };
        point_rnn[(int) current_index] = rnn;
    };
    
    auto get_rnn = [&] (){
        auto start =  chrono::high_resolution_clock::now();
        for (int current_index = 0; current_index < n; current_index++) {
            // remove current index from neighbors
            xarray<int> neighbor_indices = col(
                from_indices(argwhere(not_equal(all_point_indices, current_index))), 0
            );
            get_tiknn((int) current_index, neighbor_indices, k);
            
            // printf("%f,knn_neighbors_id,%d,,",current_time(), current_index);
            outfile << std::setprecision (16) << current_time() << ",knn_neighbors_id," <<
                    current_index << ",,";
            for(auto& i : point_knn[(int) current_index]) {outfile << i << ";"; }
            outfile << endl;
            
            // for(auto& i : point_knn[(int) current_index]) {std::cout << i << ";"; }
            // printf("\n");
            // printf("%f,|knn_neighbors|,%d,%d,\n",current_time(), current_index, (int) point_knn[(int) current_index].shape(0));
            outfile << std::setprecision (16) << current_time() << ",|knn_neighbors|," <<
                    current_index << "," << (int) point_knn[(int) current_index].shape(0) << "," << endl;
        };
        auto end = chrono::high_resolution_clock::now();
        duration<double> elapsed = end - start;
        
        outfile << std::setprecision (16) << current_time() << ",knn_time," <<
            ",," << elapsed.count() << ',' << endl;
        // printf("%f,knn_time,,%f,\n",current_time(), elapsed.count());
        
        start =  chrono::high_resolution_clock::now();
        for (int current_index =  0; current_index < n; current_index++) {
            get_pointwise_rnn((int) current_index);
            
            // printf("%f,rnn_neighbors_id,%d,,",current_time(), current_index);
            outfile << std::setprecision (16) << current_time() << ",rnn_neighbors_id," <<
                    current_index << ",,";
            for(auto& i : point_knn[(int) current_index]) {outfile << i << ";"; }
            outfile << endl;
            
            // for(auto& i : point_knn[(int) current_index]) {std::cout << i << ";"; }
            // printf("\n");
            // printf("%f,|rnn_neighbors|,%d,%d,\n",current_time(), current_index, (int) point_knn[(int) current_index].shape(0));
            outfile << std::setprecision (16) << current_time() << ",|rnn_neighbors|," <<
                    current_index << "," << (int) point_knn[(int) current_index].shape(0) << "," << endl;
        };
        end = chrono::high_resolution_clock::now();
        elapsed = end - start;
        outfile << std::setprecision (16) << current_time() << ",rnn_time," <<
            "," << elapsed.count() << ',' << endl;
    };

    // extend cluster
    function < void( int ) > search = [&] (int current_index) -> void {
        if (point_rnn[(int) current_index].size() <  k) {
            state[(int) current_index] = VISITED;
        } else {
            state[(int) current_index] = CLUSTERED;
            cluster[(int) current_index] = cluster_id;
            for (auto& neighbor_index : point_rnn[(int) current_index]) {
                if (state[(int) neighbor_index] == NOT_VISITED) {
                    search(neighbor_index);
                };
                state[(int) neighbor_index] = CLUSTERED;
                cluster[(int) neighbor_index] = cluster_id;
            };
        };
    };
    
    get_rnn();
    
    // visit all points
    while (any(equal(state, NOT_VISITED))){
        xarray<int> idx = from_indices(argwhere(equal(state, NOT_VISITED)));
        search(idx(0,0));
        cluster_id++;
    }

    // clusterize all outlier points to nearest cluster 
    while (any(equal(state, VISITED))){
        xarray<int> idx_not_clustered = from_indices(argwhere(equal(state, VISITED)));
        xarray<int> idx_clustered = from_indices(argwhere(equal(state, CLUSTERED)));
        get_knn((int) idx_not_clustered(0,0), idx_clustered, 1);
        auto closest_clustered_idx = point_knn[(int) idx_not_clustered(0,0)](0);
        cluster[(int) idx_not_clustered(0,0)] = cluster[(int) closest_clustered_idx];
        state[(int) idx_not_clustered(0,0)] = CLUSTERED;
    }
    outfile << std::setprecision (16) << current_time() << ",stop log,,," << endl;
    CLUSTER = cluster;
    STATE = state;
    // return xtuple(cluster, );
}

In [93]:
clustering_algorithm(X, 7);

## Save output to csv

In [94]:
start = chrono::high_resolution_clock::now();

ofstream out_file;
out_file.open("../out/dbscanrn_opt_cpp_flame_dataset.csv");
dump_csv(out_file, stack(xtuple(CLUSTER, STATE), 1));
out_file.close();

duration<double> elapsed = (chrono::high_resolution_clock::now() - start);
outfile << std::setprecision (16) << current_time() << ",writing_data," <<
    "," << elapsed.count() << "," << endl;
outfile.close();

# R15 dataset

In [107]:
ofstream outfile;
outfile.open("../out/LOG_dbscanrn_opt_cpp_R15_dataset_D2_R600_k9_cosine_dissimilarity_r01.log");

auto start =  chrono::high_resolution_clock::now();

ifstream input_file;
input_file.open ("../data/R15_dataset.txt");
auto data = load_csv<double>(input_file);
input_file.close();

auto end =  chrono::high_resolution_clock::now();
duration<double> elapsed = (end - start);
outfile << std::setprecision (16) << current_time() << ",reading_data," <<
    "," << elapsed.count() << ",," << endl;

data.reshape(data.shape());
xarray<double> X = view(data, all(), range(0, 2));

In [108]:
xarray<double> CLUSTER; 
xarray<double> STATE;

In [109]:
auto clustering_algorithm(const xarray<float>& X_original, int k)
{
    outfile << std::setprecision (16) << current_time() << ",start log,,," << endl;
    
    // normalization
    auto start =  chrono::high_resolution_clock::now();
    xarray<float> X = normalize(X_original);
    auto end =  chrono::high_resolution_clock::now();
    duration<double> elapsed = (end - start);

    outfile << std::setprecision (16) << current_time() << ",normalization_time," <<
        "," << elapsed.count() << "," << endl;
    
    // each data point can be in one of 3 stages
    int NOT_VISITED = -1; // not visited point
    int VISITED = 0; // non-core point
    int CLUSTERED = 1; // core point

    // initial setup
    int n = X.shape()[0];
    xarray<double> cluster = ones<double>({n}) * -1; // cluster register
    xarray<double> state = ones<double>({n}) * NOT_VISITED; // state register
    int cluster_id = 1;
    xarray<int> all_point_indices = arange(n);
    map<int, vector<int>> point_rnn;
    map<int, xarray<int>> point_knn;
    
    xarray<int> all_point_sorted; 
    xarray<int> candidates;
    float real_max; 
    xarray<double> pessimistic_estimation;
    xarray<double> candidate_distance;
    int down_row;

    
    function < void( int ) > calc_pessimistic_estimation = [&] (
        int current_index 
    ) -> void {
        
        // choosing the next point to check if he is a better neighbor
        int new_candidate = all_point_sorted(down_row);
        bool previous_check = pessimistic_estimation(down_row) < real_max;
        down_row++;
        if (!previous_check) {
            point_knn[(int) current_index] = candidates;
            return;
        };
        
        if (previous_check) {
            float similarity = cosine_dissimilarity(
                new_candidate, all_point_sorted[current_index]
            );
            outfile << std::setprecision (16) << current_time() << ",similarity_calculation," << (int) current_index << ",1," << endl;
            if (similarity < real_max) {
                candidates = filter(candidates, not_equal(candidate_distance, real_max));
                candidates = concatenate(xtuple(candidates, xarray<int>{new_candidate}));
                candidate_distance = concatenate(xtuple(candidate_distance, xarray<double>{similarity}));
                real_max = amax(candidate_distance)(0);
                calc_pessimistic_estimation((int) current_index);
            };
        };
        calc_pessimistic_estimation((int) current_index);   
    };

    auto get_tiknn = [&] (int current_index, xarray<int> neighbor_indices, int k){
        
        auto start =  chrono::high_resolution_clock::now();
        vector<double> r_dist;
        for (auto& idx : all_point_indices) {
            r_dist.push_back(cosine_dissimilarity(row(X, idx), {0, 1}));
        };
        auto end = chrono::high_resolution_clock::now();
        duration<double> elapsed = end - start;
        
        // printf("%f,pessimistic_estimation_time,%d,%f,\n",current_time(),current_index ,elapsed.count());
        outfile << std::setprecision (16) << current_time() << ",pessimistic_estimation_time," <<
                current_index << "," << elapsed.count() << "," << endl;
     
        xarray<double> r_distance = adapt(r_dist, {r_dist.size()});
        auto current_index_r_distance = r_distance[current_index];
        
        start =  chrono::high_resolution_clock::now();
        pessimistic_estimation = abs(current_index_r_distance - r_distance);
        end = chrono::high_resolution_clock::now();
        elapsed = end - start;
        
        outfile << std::setprecision (16) << current_time() << ",dist_to_ref_point_time," <<
                current_index << "," << elapsed.count() << "," << endl;
        // printf("%f,dist_to_ref_point_time,%d,%f,\n",current_time(),current_index ,elapsed.count());
        
        start =  chrono::high_resolution_clock::now();
        xarray<int> sort_indices = argsort(pessimistic_estimation);
        all_point_sorted = index_view(all_point_indices, sort_indices); // df 
        end = chrono::high_resolution_clock::now();
        elapsed = end - start;
        
        outfile << std::setprecision (16) << current_time() << ",sorting_pessimistic_est_time," <<
                current_index << "," << elapsed.count() << "," << endl;
        // printf("%f,sorting_pessimistic_est_time,%d,%f,\n",current_time(),current_index ,elapsed.count());
        
        // selecting candidates for k - nearest neighbors
        candidates = view(all_point_sorted, range(1, k+1)); //dfx
        down_row = k + 2;
                
        // calculation of similarity for candidates
        vector<double> dist;
        for (auto& candidate_idx : candidates) {
            outfile << std::setprecision (16) << current_time() << ",similarity_calculation," << (int) current_index << ",1," << endl;
            dist.push_back(cosine_dissimilarity(row(X, current_index), row(X, candidate_idx)));
        }
        candidate_distance = adapt(dist, {dist.size()});
        real_max = amax(candidate_distance)(0);
        
        calc_pessimistic_estimation((int) current_index);
    };  
        
    auto get_knn = [&] (int current_index, xarray<int> neighbor_indices, int k){
        vector<float> neighbor_sim;
        for (auto& neighbor_index : neighbor_indices) {
            
            // outfile << std::setprecision (16) << current_time() << ",similarity_calculation," << (int) current_index << ",1," << endl;
            
            neighbor_sim.push_back(cosine_dissimilarity(row(X, neighbor_index), row(X, current_index)));
        };
        xarray<float> neighbor_similaritys = adapt(neighbor_sim, {neighbor_sim.size()});
        xarray<int> sort_indices = argsort(neighbor_similaritys);
        xarray<int> neighbor_indices_sorted = index_view(neighbor_indices, sort_indices); 
        point_knn[(int) current_index] = view(neighbor_indices_sorted, range(0, k));  
    };    
    
    auto get_pointwise_rnn = [&] (int current_index){
        vector<int> rnn;
        for (auto& neighbor_index : point_knn[(int) current_index]) {
            if ((point_knn[(int) neighbor_index], (int) current_index)) {
                rnn.push_back((int) neighbor_index);
            };
        };
        point_rnn[(int) current_index] = rnn;
    };
    
    auto get_rnn = [&] (){
        auto start =  chrono::high_resolution_clock::now();
        for (int current_index = 0; current_index < n; current_index++) {
            // remove current index from neighbors
            xarray<int> neighbor_indices = col(
                from_indices(argwhere(not_equal(all_point_indices, current_index))), 0
            );
            get_tiknn((int) current_index, neighbor_indices, k);
            
            // printf("%f,knn_neighbors_id,%d,,",current_time(), current_index);
            outfile << std::setprecision (16) << current_time() << ",knn_neighbors_id," <<
                    current_index << ",,";
            for(auto& i : point_knn[(int) current_index]) {outfile << i << ";"; }
            outfile << endl;
            
            // for(auto& i : point_knn[(int) current_index]) {std::cout << i << ";"; }
            // printf("\n");
            // printf("%f,|knn_neighbors|,%d,%d,\n",current_time(), current_index, (int) point_knn[(int) current_index].shape(0));
            outfile << std::setprecision (16) << current_time() << ",|knn_neighbors|," <<
                    current_index << "," << (int) point_knn[(int) current_index].shape(0) << "," << endl;
        };
        auto end = chrono::high_resolution_clock::now();
        duration<double> elapsed = end - start;
        
        outfile << std::setprecision (16) << current_time() << ",knn_time," <<
            ",," << elapsed.count() << ',' << endl;
        // printf("%f,knn_time,,%f,\n",current_time(), elapsed.count());
        
        start =  chrono::high_resolution_clock::now();
        for (int current_index =  0; current_index < n; current_index++) {
            get_pointwise_rnn((int) current_index);
            
            // printf("%f,rnn_neighbors_id,%d,,",current_time(), current_index);
            outfile << std::setprecision (16) << current_time() << ",rnn_neighbors_id," <<
                    current_index << ",,";
            for(auto& i : point_knn[(int) current_index]) {outfile << i << ";"; }
            outfile << endl;
            
            // for(auto& i : point_knn[(int) current_index]) {std::cout << i << ";"; }
            // printf("\n");
            // printf("%f,|rnn_neighbors|,%d,%d,\n",current_time(), current_index, (int) point_knn[(int) current_index].shape(0));
            outfile << std::setprecision (16) << current_time() << ",|rnn_neighbors|," <<
                    current_index << "," << (int) point_knn[(int) current_index].shape(0) << "," << endl;
        };
        end = chrono::high_resolution_clock::now();
        elapsed = end - start;
        outfile << std::setprecision (16) << current_time() << ",rnn_time," <<
            "," << elapsed.count() << ',' << endl;
    };

    // extend cluster
    function < void( int ) > search = [&] (int current_index) -> void {
        if (point_rnn[(int) current_index].size() <  k) {
            state[(int) current_index] = VISITED;
        } else {
            state[(int) current_index] = CLUSTERED;
            cluster[(int) current_index] = cluster_id;
            for (auto& neighbor_index : point_rnn[(int) current_index]) {
                if (state[(int) neighbor_index] == NOT_VISITED) {
                    search(neighbor_index);
                };
                state[(int) neighbor_index] = CLUSTERED;
                cluster[(int) neighbor_index] = cluster_id;
            };
        };
    };
    
    get_rnn();
    
    // visit all points
    while (any(equal(state, NOT_VISITED))){
        xarray<int> idx = from_indices(argwhere(equal(state, NOT_VISITED)));
        search(idx(0,0));
        cluster_id++;
    }

    // clusterize all outlier points to nearest cluster 
    while (any(equal(state, VISITED))){
        xarray<int> idx_not_clustered = from_indices(argwhere(equal(state, VISITED)));
        xarray<int> idx_clustered = from_indices(argwhere(equal(state, CLUSTERED)));
        get_knn((int) idx_not_clustered(0,0), idx_clustered, 1);
        auto closest_clustered_idx = point_knn[(int) idx_not_clustered(0,0)](0);
        cluster[(int) idx_not_clustered(0,0)] = cluster[(int) closest_clustered_idx];
        state[(int) idx_not_clustered(0,0)] = CLUSTERED;
    }
    outfile << std::setprecision (16) << current_time() << ",stop log,,," << endl;
    CLUSTER = cluster;
    STATE = state;
    // return xtuple(cluster, );
}

In [110]:
clustering_algorithm(X, 9);

## Save output to csv

In [111]:
start = chrono::high_resolution_clock::now();

ofstream out_file;
out_file.open("../out/dbscanrn_opt_cpp_R15_dataset.csv");
dump_csv(out_file, stack(xtuple(CLUSTER, STATE), 1));
out_file.close();

duration<double> elapsed = (chrono::high_resolution_clock::now() - start);
outfile << std::setprecision (16) << current_time() << ",writing_data," <<
    "," << elapsed.count() << "," << endl;
outfile.close();